# Semi supervised learning on CIFAR10 using Pseudo-labeling





In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, ConcatDataset
import numpy as np
from tqdm import tqdm


# Data transformation and loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split the full dataset into labeled and unlabeled
num_labeled = 4000  # Number of labeled examples
indices = np.arange(len(full_trainset))
np.random.shuffle(indices)
labeled_indices = indices[:num_labeled]
unlabeled_indices = indices[num_labeled:]

labeled_subset = Subset(full_trainset, labeled_indices)
unlabeled_subset = Subset(full_trainset, unlabeled_indices)

labeled_loader = DataLoader(labeled_subset, batch_size=64, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


# ResNet-10 model

In [2]:

# Define the basic block used in ResNet-10
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define the ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self):
        super(ResNet10, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Stack of blocks
        self.layer1 = self._make_layer(16, 1, stride=1)
        self.layer2 = self._make_layer(32, 1, stride=2)
        self.layer3 = self._make_layer(64, 1, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 10)

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        #x = self.fc(x)
        return x

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet10().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Loss: {running_loss / len(dataloader)}')


# Evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total



# Training with only labeled set

In [3]:
# for num_epochs = 40       Loss: 0.3438119628126659    Test Accuracy: 56.22%
num_epochs = 40
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_model(model, labeled_loader, optimizer, criterion)


    accuracy = evaluate_model(model, test_loader)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')



Epoch 1/40


Training: 100%|██████████| 63/63 [00:03<00:00, 19.54it/s]


Loss: 3.1539104325430736
Test Accuracy: 29.38%
Epoch 2/40


Training: 100%|██████████| 63/63 [00:01<00:00, 36.09it/s]


Loss: 2.6103801878671797
Test Accuracy: 38.88%
Epoch 3/40


...

Loss: 0.39843190946276225
Test Accuracy: 56.02%
Epoch 39/40


Training: 100%|██████████| 63/63 [00:01<00:00, 33.53it/s]


Loss: 0.3572724594010247
Test Accuracy: 56.08%
Epoch 40/40


Training: 100%|██████████| 63/63 [00:01<00:00, 38.25it/s]


Loss: 0.3438119628126659
Test Accuracy: 56.22%


In [3]:
import copy
#model_copy = copy.deepcopy(model)
#model = copy.deepcopy(model_copy)


# Pseudo-labeling

In [13]:
import torch
from tqdm import tqdm
import random
from torch.utils.data import ConcatDataset, TensorDataset

def generate_pseudo_labels(model, dataloader, device, threshold=0.5):
    """
    - threshold: Minimum confidence level to keep a pseudo-label.

    """
    model.eval()
    pseudo_images = []
    pseudo_labels = []

    with torch.no_grad():
        for images, _ in tqdm(dataloader, desc="Generating Pseudo-Labels"):
            images = images.to(device)
            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)  # Get probabilities
            max_probs, predicted = torch.max(probabilities, 1)  # Get predicted class and its probability

            # Filter out predictions based on the threshold
            high_confidence_indices = max_probs > threshold
            filtered_images = images[high_confidence_indices]
            filtered_labels = predicted[high_confidence_indices]

            pseudo_images.extend(filtered_images.cpu())
            pseudo_labels.extend(filtered_labels.cpu())

    # Convert lists to tensors
    pseudo_images = torch.stack(pseudo_images)
    pseudo_labels = torch.tensor(pseudo_labels, dtype=torch.long)

    return pseudo_images, pseudo_labels



class CustomDataLoader:
    def __init__(self, dataset, batch_size=64, shuffle=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_samples = len(dataset)
        self.indices = list(range(self.num_samples))
        if self.shuffle:
            random.shuffle(self.indices)

    def __iter__(self):
        for start_idx in range(0, self.num_samples, self.batch_size):
            end_idx = min(start_idx + self.batch_size, self.num_samples)
            batch_indices = self.indices[start_idx:end_idx]
            batch_data =   [self.dataset[idx][0] for idx in batch_indices]
            batch_label = [self.dataset[idx][1] for idx in batch_indices]
            yield  torch.stack(batch_data) , torch.tensor(batch_label, dtype=torch.long)
    def __len__(self):
        return (self.num_samples + self.batch_size - 1) // self.batch_size



# Training with pseudo-labeling

In [20]:
# Training with pseudo-labeling
num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_model(model, labeled_loader, optimizer, criterion)

    # Generate pseudo-labels
    #pseudo_images, pseudo_labels = generate_pseudo_labels(model, unlabeled_loader)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pseudo_images, pseudo_labels = generate_pseudo_labels(model, unlabeled_loader, device, threshold=0.5)


    # Create a new dataset with pseudo-labeled data
    pseudo_dataset = torch.utils.data.TensorDataset(pseudo_images, pseudo_labels)
    combined_dataset = ConcatDataset([labeled_subset, pseudo_dataset])
    combined_loader = CustomDataLoader(combined_dataset, batch_size=64, shuffle=True)

    # Retrain the model with both labeled and pseudo-labeled data
    train_model(model, combined_loader, optimizer, criterion)
    accuracy = evaluate_model(model, test_loader)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10


Training: 100%|██████████| 63/63 [00:01<00:00, 36.02it/s]


Loss: 0.27919450733396745


Training: 100%|██████████| 516/516 [00:06<00:00, 76.87it/s]


Loss: 0.3514586595255275
Test Accuracy: 57.04%
Epoch 2/10


...

Loss: 0.36287295976851847
Test Accuracy: 57.72%
Epoch 10/10


Training: 100%|██████████| 63/63 [00:01<00:00, 34.34it/s]


Loss: 0.2734875733417178


Training: 100%|██████████| 517/517 [00:06<00:00, 78.73it/s]


Loss: 0.3562145724863794
Test Accuracy: 57.37%
